In [119]:
import os
import urllib
import requests
import time
import datetime

In [7]:
SKYSCANNER_API_KEY = os.environ.get('SKYSCANNER_API_KEY', 'prtl6749387986743898559646983194')

In [128]:
def do_request(request_type, url, data):
    def get_method(request_type):
        return getattr(requests, request_type)
    
    headers = {}
    
    if request_type != 'get':
        data['apikey'] = SKYSCANNER_API_KEY
        headers['content-type'] = 'application/x-www-form-urlencoded'
    else:
        url = add_apikey(url)
    
    print "\t[REQ] %s" % (url)
    
    payload = urllib.urlencode(data, doseq=True)
    r = get_method(request_type)(url, data=payload, headers=headers)
    return r

def add_apikey(url):
    return url + '?apikey=' + SKYSCANNER_API_KEY

def parse_request(r):
    if r.status_code == 201:
        return do_request('get', r.headers['Location'], {})
    
    return {'status_code': 500}

In [191]:
def fetch_data(origin, destination, when, tries=3, fail_wait=60):
    current_try = 0
    while current_try < tries:
        data = {
            'cabinclass': 'Economy',
            'country': 'ES',
            'currency': 'EUR',
            'locale': 'en-GB',
            'locationSchema': 'iata',
            'originplace': origin,
            'destinationplace': destination,
            'outbounddate': when,
            'adults': '1',
            'children': '0',
            'infants': '0'
        }
            
        r = do_request('post', 'http://partners.api.skyscanner.net/apiservices/pricing/v1.0', data)
        r = parse_request(r)
        
        if r.status_code == 200:
            return r.text.encode('utf-8')
        
        current_try += 1
        print "\t[FAIL] Reattempting in %d seconds (reason: %d)" % (fail_wait, r.status_code)
        time.sleep(fail_wait)
        
    return None

In [192]:
def get_date(fmt, ts=time.time()):
    date = datetime.datetime.fromtimestamp(ts).strftime(fmt)
    return date

def fetch_and_save(origin, destination, when, tries=3, fail_wait=60):
    print "[DO] %s to %s on %s" % (origin, destination, when)
    
    date = get_date('%Y-%m-%d-%H')
    filename = '%s_%s_%s_%s.json' % (origin, destination, date, when)
    filename = os.path.join('flights-data', filename)
    
    try:
        os.makedirs('flights-data')
    except:
        pass
    
    data = fetch_data(origin, destination, when, tries, fail_wait)
    if data is None:
        print "[ERROR] %s to %s on %s" % (origin, destination, when)
        
    with open(filename, 'w') as fp:
        fp.write(data)
        
    print "[SAVED] %s to %s on %s" % (origin, destination, when)

In [193]:
fetch_and_save('BCN', 'PMI', '2017-07-20')

[DO] BCN to PMI on 2017-07-20
	[REQ] http://partners.api.skyscanner.net/apiservices/pricing/v1.0
	[REQ] http://partners.api.skyscanner.net/apiservices/pricing/uk1/v1.0/a4f0b7ff27f9404fbab7d56250d77f8a_rrsqbjcb_06a13f0a788e803fcc56e78802891a26?apikey=prtl6749387986743898559646983194
[SAVED] BCN to PMI on 2017-07-20


In [194]:
def start_fetching(look_ahead_days, itineraries, start=int(time.time())):
    DAY = 60 * 60 * 24
    LOOK_AHEAD = look_ahead_days * DAY
    TARGET_TIME = start + LOOK_AHEAD

    now = int(time.time())
    start = now + LOOK_AHEAD
    target = TARGET_TIME
    
    for _ in range(2):
        print "-------------------------------------"
        print "[UPDATE] Fetching day %d" % ((now + LOOK_AHEAD + DAY - start) / DAY)

        for ts in range(target, now + LOOK_AHEAD + DAY, DAY):
            for origin, destination in itineraries:
                fetch_and_save(origin, destination, get_date('%Y-%m-%d', ts))

        if now + LOOK_AHEAD + DAY - target >= LOOK_AHEAD:
            target += DAY

        now += DAY

In [195]:
start_fetching(5, (('BCN', 'PMI'),))

-------------------------------------
[UPDATE] Fetching day 1
[DO] BCN to PMI on 2017-02-27
	[REQ] http://partners.api.skyscanner.net/apiservices/pricing/v1.0
	[REQ] http://partners.api.skyscanner.net/apiservices/pricing/uk1/v1.0/f61ed9f654934878b5833fe4d0c92976_rrsqbjcb_06a13f0a788e803fcc56e78802891a26?apikey=prtl6749387986743898559646983194
	[FAIL] Reattempting in 60 seconds (reason: 304)
	[REQ] http://partners.api.skyscanner.net/apiservices/pricing/v1.0
	[REQ] http://partners.api.skyscanner.net/apiservices/pricing/uk1/v1.0/01ccf8163c8243089c68978d42a2a2ea_rrsqbjcb_06a13f0a788e803fcc56e78802891a26?apikey=prtl6749387986743898559646983194
[SAVED] BCN to PMI on 2017-02-27
[DO] BCN to PMI on 2017-02-28
	[REQ] http://partners.api.skyscanner.net/apiservices/pricing/v1.0
	[REQ] http://partners.api.skyscanner.net/apiservices/pricing/uk1/v1.0/f03fe4d5108b41ef8f805b30430b325e_rrsqbjcb_06a13f0a788e803fcc56e78802891a26?apikey=prtl6749387986743898559646983194
[SAVED] BCN to PMI on 2017-02-28
----